In [1]:
#model套件
import torch
from torch import nn as nn
#資料處理套件
import numpy as np
#時間套件
import time
#列舉套件
from enum import Enum

import sys
#將父目錄添加到模組搜尋路徑中
sys.path.append('..')
#取得模型
import GetModel as GM
#訓練/測試 模型
import TrainModel as TM
#資料前處理
import TrainDataPreprocessing as TDP

In [2]:
#存檔位置
ModelMode = "ESC50_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
sMode = 4
filePath_Train = ModelMode + TrainDataSoundMode[sMode] + "_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[sMode] +  "_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[sMode]

In [3]:
#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
data_1S = TDP.loadDataESC50(soundLength=1)
print("讀取3秒資料")
data_3S = TDP.loadDataESC50(soundLength=3)
print("讀取5秒資料")
data_5S = TDP.loadDataESC50(soundLength=5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))

讀取1秒資料


D:\Hu_code\ResNet18_ESC50\ESC50_kFold_531s\..\TrainDataPreprocessing.py:57: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:248.)
  dataX_Fold = torch.Tensor(dataX_Fold)


Progress: 100% - Completed
讀取3秒資料
Progress: 100% - Completed
讀取5秒資料
Progress: 100% - Completed
Data Loading Time: 132.55 seconds


In [4]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []

#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    kTLTrainTime = []
    kTLTrainAccuracy = []
    kTLTestAccuracy1 = []
    kTLTestAccuracy3 = []
    kTLTestAccuracy5 = []

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = GM.getModel(num_classes=50)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    
    for TL in range(3):
        print("----------------------------------------------------------------------------------------------------------------")
        print("fold: ", fold, "TL: ", TL)
        print("----------------------------------------------------------------------------------------------------------------")
        
        #------------------------------------------------
        #需要調整的地方
        #------------------------------------------------
        #--要用幾秒訓練 data_1S/data_3S/data_1S
        #--學習率
        #--epoch
        if(TL==0):
            trainData = data_5S
            epoch = 90
            lr = 0.0032
        elif(TL==1):
            trainData = data_3S
            epoch = 90
            lr = 0.0032
        else:
            trainData = data_1S
            epoch = 90
            lr = 0.0032
        #------------------------------------------------
    

        #紀錄訓練開始時間
        start_time = time.time()

        #訓練模型
        model, train_loss, train_accuracy = TM.train_model_TrainAll(model=model,
                                                                inputData=trainData,
                                                                trainFold=fold,
                                                                setLR=lr,
                                                                setEpoch=epoch)
    
        #測試準確度
        test_accuracy1 = TM.test_model(model=model, inputData=data_1S, testFold=fold)
        test_accuracy3 = TM.test_model(model=model, inputData=data_3S, testFold=fold)
        test_accuracy5 = TM.test_model(model=model, inputData=data_5S, testFold=fold)

        #紀錄訓練結束時間
        end_time = time.time()

        #計算訓練耗時
        training_time = end_time - start_time

        #印出訓練耗時
        print("Training Time: {:.2f} seconds".format(training_time))

        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("訓練過程\n")
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        for i in range(len(train_loss)):
            # 寫入資料
            f.write("Fold: {} TL: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, TL, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
        f.write("Training Time: {:.2f} seconds\n".format(training_time))
        f.write("---------------------------------------------------------------------------------------------------------------\n")

        #--------------------------------------------------------------------
        
        kTLTrainTime.append(training_time)
        kTLTrainAccuracy.append(train_accuracy[-1])
        kTLTestAccuracy1.append(test_accuracy1)
        kTLTestAccuracy3.append(test_accuracy3)
        kTLTestAccuracy5.append(test_accuracy5)
        
    kFoldTrainTime.append(kTLTrainTime)
    kFoldTrainAccuracy.append(kTLTrainAccuracy)
    kFoldTestAccuracy1.append(kTLTestAccuracy1)
    kFoldTestAccuracy3.append(kTLTestAccuracy3)
    kFoldTestAccuracy5.append(kTLTestAccuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

可用的 GPU 数量: 1
----------------------------------------------------------------------------------------------------------------
fold:  1 TL:  0
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/90 Train_Loss: 3.4241397380828857 Train_acc: 9.875000%
Train Epoch: 2/90 Train_Loss: 2.8269875049591064 Train_acc: 18.875000%
Train Epoch: 3/90 Train_Loss: 2.4963057041168213 Train_acc: 31.875000%
Train Epoch: 4/90 Train_Loss: 2.0837297439575195 Train_acc: 35.250000%
Train Epoch: 5/90 Train_Loss: 1.6082035303115845 Train_acc: 43.875000%
Train Epoch: 6/90 Train_Loss: 1.2093905210494995 Train_acc: 51.375000%
Train Epoch: 7/90 Train_Loss: 1.2945507764816284 Train_acc: 56.062500%
Train Epoch: 8/90 Train_Loss: 1.3260656595230103 Train_acc: 63.062500%
Train Epoch: 9/90 Train_Loss: 1.062900424003601 Train_acc: 66.250000%
Train Epoch: 10/90 Train_Loss: 0.8705440163612366 Train_acc: 68.375000%
Train Epoch: 11/90 Train_Loss: 0.81

Train Epoch: 13/90 Train_Loss: 0.47185295820236206 Train_acc: 93.687500%
Train Epoch: 14/90 Train_Loss: 0.35345423221588135 Train_acc: 91.854172%
Train Epoch: 15/90 Train_Loss: 0.07548635452985764 Train_acc: 92.166672%
Train Epoch: 16/90 Train_Loss: 0.08856742829084396 Train_acc: 95.479172%
Train Epoch: 17/90 Train_Loss: 0.19982273876667023 Train_acc: 96.375000%
Train Epoch: 18/90 Train_Loss: 0.1258372664451599 Train_acc: 96.791672%
Train Epoch: 19/90 Train_Loss: 0.2951914370059967 Train_acc: 96.500000%
Train Epoch: 20/90 Train_Loss: 0.048684850335121155 Train_acc: 96.666672%
Train Epoch: 21/90 Train_Loss: 0.14140602946281433 Train_acc: 96.916672%
Train Epoch: 22/90 Train_Loss: 0.09128854423761368 Train_acc: 96.729172%
Train Epoch: 23/90 Train_Loss: 0.10270578414201736 Train_acc: 96.625000%
Train Epoch: 24/90 Train_Loss: 0.20274528861045837 Train_acc: 96.833336%
Train Epoch: 25/90 Train_Loss: 0.1598738431930542 Train_acc: 97.125000%
Train Epoch: 26/90 Train_Loss: 0.0013455849839374423 

Train Epoch: 31/90 Train_Loss: 0.1613103747367859 Train_acc: 93.237503%
Train Epoch: 32/90 Train_Loss: 0.15207812190055847 Train_acc: 93.200005%
Train Epoch: 33/90 Train_Loss: 0.0003776683588512242 Train_acc: 93.237503%
Train Epoch: 34/90 Train_Loss: 0.3179548382759094 Train_acc: 93.237503%
Train Epoch: 35/90 Train_Loss: 0.35746195912361145 Train_acc: 93.250008%
Train Epoch: 36/90 Train_Loss: 0.31559696793556213 Train_acc: 93.262505%
Train Epoch: 37/90 Train_Loss: 0.3710593283176422 Train_acc: 93.250008%
Train Epoch: 38/90 Train_Loss: 0.12032786011695862 Train_acc: 93.250008%
Train Epoch: 39/90 Train_Loss: 0.471647173166275 Train_acc: 93.250008%
Train Epoch: 40/90 Train_Loss: 0.36888912320137024 Train_acc: 93.262505%
Train Epoch: 41/90 Train_Loss: 0.20403659343719482 Train_acc: 93.262505%
Train Epoch: 42/90 Train_Loss: 0.22047901153564453 Train_acc: 93.250008%
Train Epoch: 43/90 Train_Loss: 0.23897549510002136 Train_acc: 93.250008%
Train Epoch: 44/90 Train_Loss: 0.1709507405757904 Trai

Train Epoch: 48/90 Train_Loss: 0.009421994909644127 Train_acc: 100.000000%
Train Epoch: 49/90 Train_Loss: 0.0027589895762503147 Train_acc: 100.000000%
Train Epoch: 50/90 Train_Loss: 0.0016640323447063565 Train_acc: 100.000000%
Train Epoch: 51/90 Train_Loss: 0.0015492980601266026 Train_acc: 100.000000%
Train Epoch: 52/90 Train_Loss: 0.0019763747695833445 Train_acc: 100.000000%
Train Epoch: 53/90 Train_Loss: 0.0068464577198028564 Train_acc: 100.000000%
Train Epoch: 54/90 Train_Loss: 0.0014522101264446974 Train_acc: 100.000000%
Train Epoch: 55/90 Train_Loss: 0.0017099183751270175 Train_acc: 100.000000%
Train Epoch: 56/90 Train_Loss: 0.002401805017143488 Train_acc: 100.000000%
Train Epoch: 57/90 Train_Loss: 0.0014391079312190413 Train_acc: 100.000000%
Train Epoch: 58/90 Train_Loss: 0.006941706407815218 Train_acc: 100.000000%
Train Epoch: 59/90 Train_Loss: 0.0010626594303175807 Train_acc: 100.000000%
Train Epoch: 60/90 Train_Loss: 0.0015441881259903312 Train_acc: 100.000000%
Train Epoch: 61

Train Epoch: 64/90 Train_Loss: 0.026564450934529305 Train_acc: 97.395836%
Train Epoch: 65/90 Train_Loss: 0.11063671112060547 Train_acc: 97.395836%
Train Epoch: 66/90 Train_Loss: 0.02496962435543537 Train_acc: 97.395836%
Train Epoch: 67/90 Train_Loss: 0.16628675162792206 Train_acc: 97.395836%
Train Epoch: 68/90 Train_Loss: 0.09423210471868515 Train_acc: 97.395836%
Train Epoch: 69/90 Train_Loss: 0.07381410896778107 Train_acc: 97.395836%
Train Epoch: 70/90 Train_Loss: 0.07577033340930939 Train_acc: 97.395836%
Train Epoch: 71/90 Train_Loss: 0.1312258243560791 Train_acc: 97.395836%
Train Epoch: 72/90 Train_Loss: 0.07949304580688477 Train_acc: 97.395836%
Train Epoch: 73/90 Train_Loss: 0.11056364327669144 Train_acc: 97.395836%
Train Epoch: 74/90 Train_Loss: 0.26523202657699585 Train_acc: 97.395836%
Train Epoch: 75/90 Train_Loss: 0.06633123755455017 Train_acc: 97.395836%
Train Epoch: 76/90 Train_Loss: 0.06645213812589645 Train_acc: 97.395836%
Train Epoch: 77/90 Train_Loss: 0.0825352892279625 T

...

Train Epoch: 38/90 Train_Loss: 0.005366107914596796 Train_acc: 100.000000%
Train Epoch: 39/90 Train_Loss: 0.0035453285090625286 Train_acc: 100.000000%
Train Epoch: 40/90 Train_Loss: 0.005127908196300268 Train_acc: 100.000000%
Train Epoch: 41/90 Train_Loss: 0.00907884817570448 Train_acc: 100.000000%
Train Epoch: 42/90 Train_Loss: 0.005944449454545975 Train_acc: 100.000000%
Train Epoch: 43/90 Train_Loss: 0.004868166521191597 Train_acc: 99.875000%
Train Epoch: 44/90 Train_Loss: 0.03038099594414234 Train_acc: 99.937500%
Train Epoch: 45/90 Train_Loss: 0.0031686972361057997 Train_acc: 99.937500%
Train Epoch: 46/90 Train_Loss: 0.003041299292817712 Train_acc: 100.000000%
Train Epoch: 47/90 Train_Loss: 0.004083888605237007 Train_acc: 100.000000%
Train Epoch: 48/90 Train_Loss: 0.004106726963073015 Train_acc: 100.000000%
Train Epoch: 49/90 Train_Loss: 0.003948481287807226 Train_acc: 100.000000%
Train Epoch: 50/90 Train_Loss: 0.001997855491936207 Train_acc: 100.000000%
Train Epoch: 51/90 Train_Los

Train Epoch: 54/90 Train_Loss: 0.07863304018974304 Train_acc: 97.125000%
Train Epoch: 55/90 Train_Loss: 0.18340538442134857 Train_acc: 97.125000%
Train Epoch: 56/90 Train_Loss: 0.0003270564484409988 Train_acc: 97.166672%
Train Epoch: 57/90 Train_Loss: 0.07558728009462357 Train_acc: 97.208336%
Train Epoch: 58/90 Train_Loss: 0.22339114546775818 Train_acc: 97.208336%
Train Epoch: 59/90 Train_Loss: 0.18284659087657928 Train_acc: 97.208336%
Train Epoch: 60/90 Train_Loss: 0.11045555025339127 Train_acc: 97.229172%
Train Epoch: 61/90 Train_Loss: 0.00022765411995351315 Train_acc: 97.208336%
Train Epoch: 62/90 Train_Loss: 0.14809639751911163 Train_acc: 97.187500%
Train Epoch: 63/90 Train_Loss: 0.06602410972118378 Train_acc: 97.166672%
Train Epoch: 64/90 Train_Loss: 0.04430104419589043 Train_acc: 97.208336%
Train Epoch: 65/90 Train_Loss: 0.00042978290002793074 Train_acc: 97.187500%
Train Epoch: 66/90 Train_Loss: 0.08541353046894073 Train_acc: 97.208336%
Train Epoch: 67/90 Train_Loss: 0.1703635305

Train Epoch: 72/90 Train_Loss: 0.07068582624197006 Train_acc: 92.800003%
Train Epoch: 73/90 Train_Loss: 0.47231924533843994 Train_acc: 92.812508%
Train Epoch: 74/90 Train_Loss: 0.29691362380981445 Train_acc: 92.825005%
Train Epoch: 75/90 Train_Loss: 0.09909942001104355 Train_acc: 92.825005%
Train Epoch: 76/90 Train_Loss: 0.22164195775985718 Train_acc: 92.812508%
Train Epoch: 77/90 Train_Loss: 0.10498946160078049 Train_acc: 92.825005%
Train Epoch: 78/90 Train_Loss: 0.32870686054229736 Train_acc: 92.812508%
Train Epoch: 79/90 Train_Loss: 0.17942412197589874 Train_acc: 92.825005%
Train Epoch: 80/90 Train_Loss: 0.48273903131484985 Train_acc: 92.837502%
Train Epoch: 81/90 Train_Loss: 0.2919875383377075 Train_acc: 92.812508%
Train Epoch: 82/90 Train_Loss: 0.13030605018138885 Train_acc: 92.812508%
Train Epoch: 83/90 Train_Loss: 0.329078733921051 Train_acc: 92.825005%
Train Epoch: 84/90 Train_Loss: 0.237104132771492 Train_acc: 92.837502%
Train Epoch: 85/90 Train_Loss: 0.17807264626026154 Train

Train Epoch: 88/90 Train_Loss: 0.0013540079817175865 Train_acc: 100.000000%
Train Epoch: 89/90 Train_Loss: 0.0014819703064858913 Train_acc: 100.000000%
Train Epoch: 90/90 Train_Loss: 0.0037316791713237762 Train_acc: 100.000000%
Test accuracy of the model: 30.650000%
Test accuracy of the model: 57.166667%
Test accuracy of the model: 65.250000%
Training Time: 155.58 seconds
----------------------------------------------------------------------------------------------------------------
fold:  5 TL:  1
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/90 Train_Loss: 0.8432325720787048 Train_acc: 70.729172%
Train Epoch: 2/90 Train_Loss: 0.7317202687263489 Train_acc: 81.270836%
Train Epoch: 3/90 Train_Loss: 0.5175936818122864 Train_acc: 85.395836%
Train Epoch: 4/90 Train_Loss: 0.3358914256095886 Train_acc: 87.645836%
Train Epoch: 5/90 Train_Loss: 0.4020340144634247 Train_acc: 90.062500%
Train Epoch: 6/90 Train_Loss

Train Epoch: 10/90 Train_Loss: 0.2892652451992035 Train_acc: 89.625008%
Train Epoch: 11/90 Train_Loss: 0.3985470235347748 Train_acc: 89.737503%
Train Epoch: 12/90 Train_Loss: 0.5807903409004211 Train_acc: 90.525002%
Train Epoch: 13/90 Train_Loss: 0.32973533868789673 Train_acc: 91.212502%
Train Epoch: 14/90 Train_Loss: 0.2205813229084015 Train_acc: 91.100006%
Train Epoch: 15/90 Train_Loss: 0.459030419588089 Train_acc: 90.087502%
Train Epoch: 16/90 Train_Loss: 0.2252691388130188 Train_acc: 91.500008%
Train Epoch: 17/90 Train_Loss: 0.2731578052043915 Train_acc: 92.312508%
Train Epoch: 18/90 Train_Loss: 0.21310101449489594 Train_acc: 92.587502%
Train Epoch: 19/90 Train_Loss: 0.32285091280937195 Train_acc: 92.512505%
Train Epoch: 20/90 Train_Loss: 0.2029675990343094 Train_acc: 92.687508%
Train Epoch: 21/90 Train_Loss: 0.1189243346452713 Train_acc: 92.650002%
Train Epoch: 22/90 Train_Loss: 0.49435028433799744 Train_acc: 92.675003%
Train Epoch: 23/90 Train_Loss: 0.312982976436615 Train_acc: 9

In [5]:
class FilterMode(Enum):
    Max = "Max"
    Min = "Min"
    Mean = "Mean"
    
def listFilter(data, mode:FilterMode):
    TL = []
    for fold in range(len(data)):
        TL.append(data[fold][len(data[fold])-1])
    
    switch = {
        FilterMode.Max: max(TL),
        FilterMode.Min: min(TL),
        FilterMode.Mean: np.mean(TL)
    }

    return switch.get(mode, -1)

#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    for TL in range(3):
        # 寫入資料
        f.write("Fold: {} TL: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, TL, kFoldTrainAccuracy[fold][TL], kFoldTestAccuracy1[fold][TL], kFoldTestAccuracy3[fold][TL], kFoldTestAccuracy5[fold][TL], kFoldTrainTime[fold][TL]))
    f.write("-----------------------------------------------------------------------------------\n")
    
# 寫入資料[]
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Max), listFilter(kFoldTestAccuracy1, FilterMode.Max), listFilter(kFoldTestAccuracy3, FilterMode.Max), listFilter(kFoldTestAccuracy5, FilterMode.Max)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Min), listFilter(kFoldTestAccuracy1, FilterMode.Min), listFilter(kFoldTestAccuracy3, FilterMode.Min), listFilter(kFoldTestAccuracy5, FilterMode.Min)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Mean), listFilter(kFoldTestAccuracy1, FilterMode.Mean), listFilter(kFoldTestAccuracy3, FilterMode.Mean), listFilter(kFoldTestAccuracy5, FilterMode.Mean)))

f.close()